In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Dense
from transformers import AutoTokenizer
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

2024-05-21 06:23:09.598385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 06:23:09.598470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 06:23:09.722791: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Пробую обучить нейросеть на Tensorflow

In [ ]:
# Sample data preparation (assuming you have your data in a pandas DataFrame)
# Replace this with your actual DataFrame
data = pd.DataFrame({
    'input_text': ['i ike eting cakes', 'i go to shool everydy'],
    'target_text': ['i like eating cakes', 'i go to school everyday']
})

data = pd.read_csv('/kaggle/input/dop-test-files/errors.csv').drop(columns=['Unnamed: 0'])
data

In [ ]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['preds'].tolist() + data['label'].tolist())

# Convert text to sequences
input_sequences = tokenizer.texts_to_sequences(data['preds'].tolist())
target_sequences = tokenizer.texts_to_sequences(data['label'].tolist())

# Padding sequences to the same length
max_seq_len = max(max(len(seq) for seq in input_sequences), max(len(seq) for seq in target_sequences))
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='post')
padded_target_sequences = pad_sequences(target_sequences, maxlen=max_seq_len, padding='post')

# Define model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 1024
rnn_units = 512

In [ ]:
input_sequences[:10]

In [ ]:
target_sequences[:10]

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_len))
model.add(LSTM(rnn_units, return_sequences=True))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Prepare target data for training
padded_target_sequences = np.expand_dims(padded_target_sequences, -1)

# Train the model
model.fit(padded_input_sequences, padded_target_sequences, epochs=50, batch_size=50)

In [ ]:
def predict(input_text):
    # Preprocess the input text
    input_seq = tokenizer.texts_to_sequences([input_text])
    padded_input_seq = pad_sequences(input_seq, maxlen=max_seq_len, padding='post')

    # Predict the output sequence
    predictions = model.predict(padded_input_seq)
    predicted_sequence = np.argmax(predictions, axis=-1)
    
    # Convert the predicted sequence back to text
    decoded_sentence = ' '.join(tokenizer.index_word.get(index, '') for index in predicted_sequence[0])
    
    return decoded_sentence.strip()

# Example prediction
corrected_text = predict('мотивация должна быыыть всегдаааа')
print('Corrected Text:', corrected_text)

# Файн тюнинг предобученного корректора

In [2]:
!pip install datasets
!apt install git-lfs
!pip install transformers
!pip install sentencepiece 
!pip install rouge_score

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 65 not upgraded.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=fa84351a195a47b8a6cd8916dd2b651da7c3677c7048ff5a5bbacc3ee4be1bf5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
# Импорт библиотек
import numpy as np
from datasets import Dataset
import tensorflow as tf
import nltk
from transformers import T5TokenizerFast, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import torch
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import load_dataset, load_metric

In [11]:
# загрузка параметров
#raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")
nltk.download('punkt')

/tmp/ipykernel_34/2633626551.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
# Определение параметров
REPO = "t5-russian-spell"  # Введите наазвание название репозитория
MODEL_NAME = "UrukHan/t5-russian-spell" # Введите наазвание выбранной модели из хаба
MAX_INPUT = 256  # Введите максимальную длинну входных данных  в токенах (длинна входных фраз в словах (можно считать полслова токен))
MAX_OUTPUT  = 256 # Введите максимальную длинну прогнозов в токенах (можно уменьшить для задач суммризации или других задач где выход короче)
BATCH_SIZE = 20 
DATASET = 'UrukHan/t5-russian-spell_I'   # Введите наазвание название датасета

# Преобразование DataFrame в Dataset
data = Dataset.from_pandas(df)

# Разделение DataFrame на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=0.2)

# Преобразование обратно в Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [18]:
# Загрузка модели и токенизатора
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

model.config.max_length = MAX_OUTPUT  # по умолчанию 20, поэтому во всех моделях прогнозы обрезаются выходные последовательности
# Закоментить после первого соъранения в репозиторий свой необъязательно

train = train_dataset
test = test_dataset.train_test_split(0.02)  # Уменьшил так тестовыу. выборку чтоб не ждать долго расчет ошибок между эпохами

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) #return_tensors="tf"

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [36]:
training_args = Seq2SeqTrainingArguments(
  output_dir = '/kaggle/working/',
  #overwrite_output_dir=True,
  evaluation_strategy='steps',
  #learning_rate=2e-5,
  eval_steps=5000,
  save_steps=5000,
  num_train_epochs=10,
  predict_with_generate=True,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE,
  fp16=True,
  save_total_limit=2,
  #generation_max_length=256,
  #generation_num_beams=4,
  weight_decay=0.005,
  #logging_dir='logs',
  push_to_hub=False,
  report_to="none"
)

In [37]:
# Выберем вручную оптимизатор. Т5 в оригинальной архитектуре использует Адафактор оптимизатор
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False,
)
lr_scheduler = AdafactorSchedule(optimizer)

In [33]:
!pip install wandb
import os
#os.environ['WANDB_DISABLED'] = 'True'
#os.environ["WANDB_SILENT"] = "true"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset = train,
  eval_dataset = test,
  optimizers = (optimizer, lr_scheduler),
  tokenizer = tokenizer,
  compute_metrics=compute_metrics
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

# Пробую обучить t5 трансформер на 60 миллионов параметров

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

df = pd.read_csv('/kaggle/input/dop-test-files/errors.csv').drop(columns=['Unnamed: 0'])
df.head(5)

2024-05-28 06:30:07.463619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 06:30:07.463720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 06:30:07.590886: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,label,preds
0,иностранный агент,иностранный аген
1,свидетельствуют о проблемах с печенью,свидетельствуют о праблемах спецнью
2,найдите способ быть полезными другим людям,найдиче сьпособ быть полезном другим людем
3,я уже поставил белье в стирку,я уже поставил бельо встиру
4,круглый мяч,круглый мядчь


In [2]:
X_train, X_test, y_train, y_test = train_test_split(df['preds'], df['label'], test_size=0.3, random_state=42)

df_train = pd.DataFrame({'preds': X_train, 'label': y_train})
df_val = pd.DataFrame({'preds': X_test, 'label': y_test})

In [3]:
# Convert the DataFrame to a Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

# Combine them into a DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Preprocessing function to tokenize the data
def preprocess_function(examples):
    inputs = ['label: ' + inp for inp in examples['preds']]
    targets = [tgt for tgt in examples['label']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length').input_ids
    model_inputs['label'] = labels
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/1036 [00:00<?, ? examples/s]

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

In [5]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.4 MB/s eta 0:00:0000:0100:01


In [6]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=30,
    weight_decay=0.01
)

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.095833
2,No log,0.078306
3,No log,0.058733
4,No log,0.046769
5,No log,0.043604
6,No log,0.042068
7,No log,0.041437
8,No log,0.040771
9,No log,0.040598
10,0.378800,0.040271


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1560, training_loss=0.14902085050558433, metrics={'train_runtime': 1871.3936, 'train_samples_per_second': 16.608, 'train_steps_per_second': 0.834, 'total_flos': 4206423189749760.0, 'train_loss': 0.14902085050558433, 'epoch': 30.0})

In [8]:
# Fine-tune the model
#trainer.train()

# Save the model and tokenizer
model.save_pretrained('t5_small_fine_tuned_30_epochs_model')
tokenizer.save_pretrained('t5_small_fine_tuned_30_epochs_tokenizer')

('t5_small_fine_tuned_30_epochs_tokenizer/tokenizer_config.json',
 't5_small_fine_tuned_30_epochs_tokenizer/special_tokens_map.json',
 't5_small_fine_tuned_30_epochs_tokenizer/spiece.model',
 't5_small_fine_tuned_30_epochs_tokenizer/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./saved_model')
tokenizer = T5Tokenizer.from_pretrained('./saved_model')

# Function to correct mistakes
def correct_mistakes(text, model, tokenizer):
    # Prepare the input text
    input_text = "correct: " + text
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    
    # Generate the corrected text
    outputs = model.generate(inputs, max_length=512, num_beams=5, early_stopping=True)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return corrected_text

# Example usage
text = "Привет, как дила?"
corrected_text = correct_mistakes(text, model, tokenizer)
print(corrected_text)  # Output should be "Привет, как дела?"